# Criar VENV

https://oandersonbm.medium.com/usando-python-no-vs-code-com-ambiente-virtual-venv-ecef7959b652

In [39]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import locale

# Definindo a configuração local para o Brasil (pt_BR)
locale.setlocale(locale.LC_ALL, 'pt_BR')

# Personalizando para ver todas as colunas e linhas do DataFrame
pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None) 

# Base de dados

In [40]:
# Realizando a junção dos 3 arquivos .CSV num único DataFrame

# Caminhos para os arquivos .CSV
caminho_1 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini.csv"

caminho_2 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(1).csv"

caminho_3 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(2).csv"

caminho_4 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(3).csv"

caminho_5 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(4).csv"

caminho_6 = r"C:\Users\User\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(5).csv"



# Leitura dos arquivos .CSV em DataFrames individuais
df1 = pd.read_csv(caminho_1)
df2 = pd.read_csv(caminho_2)
df3 = pd.read_csv(caminho_3)
df4 = pd.read_csv(caminho_4)
df5 = pd.read_csv(caminho_5)
df6 = pd.read_csv(caminho_6)

# Concatenação dos DataFrames em um único DataFrame
df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

# Excluindo linhas que contenham a string "Tela" em qualquer coluna
df = df[~df.apply(lambda row: row.astype(str).str.contains('Tela')).any(axis=1)]

# Transformando o tipo de dado das colunas de valores monetários
df['A_LIQUIDAR'] = df['A_LIQUIDAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['LIQUIDADO_A_PAGAR'] = df['LIQUIDADO_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['TOTAL_A_PAGAR'] = df['TOTAL_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['PAGO'] = df['PAGO'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

# Convertendo a coluna DATA para datetime
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%y')

# Convertendo a coluna DIAS em número inteiro
df['DIAS'] = df['DIAS'].astype(int)



# Trabalhando com um DATAFRAME voltado para controle dos SALDOS

In [41]:
df_saldos = df[['SIGLA UG', 'ANO', 'NOME_CREDOR', 'DATA', 'DIAS', 'ND','NDSI', 'NOME_NDSI', 'NE', 'PI', 'NOME_PI','TIPO', 'NOME_UGR', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']]

In [42]:
# Saldos Totais
total_pago = df['PAGO'].sum().round(2)
total_a_liquidar = df['A_LIQUIDAR'].sum().round(2)
total_liquidado_a_pagar = df['LIQUIDADO_A_PAGAR'].sum().round(2)
total_a_pagar = df['TOTAL_A_PAGAR'].sum().round(2)


print(f'O total pago pela ESA até o momento foi: {locale.currency(total_pago, grouping=True)}')
print(f'O total de saldo a liquidar pela ESA até o momento foi: {locale.currency(total_a_liquidar, grouping=True)}')
print(f'O total de saldo liquidado a pagar pela ESA até o momento foi: {locale.currency(total_liquidado_a_pagar, grouping=True)}')
print(f'O total de saldo a pagar pela ESA até o momento foi: {locale.currency(total_a_pagar, grouping=True)}')

O total pago pela ESA até o momento foi: R$ 39.009.228,20
O total de saldo a liquidar pela ESA até o momento foi: R$ 4.729.079,29
O total de saldo liquidado a pagar pela ESA até o momento foi: R$ 33.001,61
O total de saldo a pagar pela ESA até o momento foi: R$ 4.762.080,90


In [43]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ND', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ND',
             labels={'ND':'Natureza da Despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=500,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [44]:
# Saldo a liquidar por Subitem
cols = ['NDSI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NDSI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NDSI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por NDSI',
             labels={'NDSI':'Subitem da despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=600,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [45]:
# Saldo a liquidar por tipo de PI
cols = ['NOME_PI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_PI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_PI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por PI',
             labels={'NOME_PI':'Tipo de PI', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=800,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [46]:
# Saldo a liquidar por tipo de Empenho
cols = ['TIPO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por tipo de empenho',
             labels={'TIPO':'Tipo de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [47]:
# Saldo a liquidar por ano
cols = ['ANO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ANO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ANO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'ANO':'Ano do Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [48]:
# Saldo a liquidar por tipo de NE
cols = ['TIPO_NE', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO_NE').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO_NE', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'TIPO_NE':'Tipo de Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [49]:
# Saldo a liquidar por tipo de NE
cols = ['NOME_UGR', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_UGR').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_UGR', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'NOME_UGR':'Nome das UGR', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [50]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'PAGO']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
# Criando o gráfico
fig = go.Figure()

# Adicionando as colunas y1, y2 e y3 no eixo y
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['A_LIQUIDAR'], name='A Liquidar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['LIQUIDADO_A_PAGAR'], name='Liquidado a pagar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['PAGO'], name='Pago'))

fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")
fig.update_yaxes(type="log")

fig.show()

# Suporte para o Dashboard

In [51]:
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']

df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
df_aux

,NE,A_LIQUIDAR,DIAS,NOME_NDSI,NDSI
2995,2022NE001087,1598.00,449,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,33904012
2796,2022NE002483,212239.40,380,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
2795,2022NE002482,155796.97,380,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
3012,2022NE002702,12733.76,366,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
328,2023NE000001,0.30,353,SERVICOS DE ENERGIA ELETRICA,33903943
5156,2023NE000002,111.93,352,"SERV.MEDICO-HOSPITAL.,ODONTOL.E LABORATORIAIS",33903950
3099,2023NE000033,22498.10,329,PASSAGENS PARA O PAIS,33903301
3167,2023NE000034,9817.75,329,PASSAGENS PARA O PAIS,33903301
3052,2023NE000157,4047.13,317,SERVICOS DOMESTICOS,33903946
3057,2023NE000195,715.00,297,SERVICOS DE TELECOMUNICACOES,33903958


In [52]:
# Saldo a liquidar por tipo de NE
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']
df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
fig = px.bar(df_aux, 
             x='NE', 
             y='A_LIQUIDAR',
             title='10 empenhos com mais tempo na tela',
             labels={'NE':'Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             color='DIAS',
             color_continuous_scale = 'reds',
             log_y=True,
             template='plotly_dark',
             hover_name="NE", hover_data={'DIAS': True, 'A_LIQUIDAR': True, 'NOME_NDSI': True, 'NDSI': True})
fig.update_traces(textposition = 'outside')


fig.show()

## Informações para SideBar

In [53]:
# Qtde de empenhos realizados até o momento
df.shape[0]

5354

# Requisição em API para consulta a CNPJ

In [54]:
import requests
from pprint import pprint

url = "https://receitaws.com.br/v1/cnpj/03126110000192"

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

pprint(response.json())

{'abertura': '04/05/1999',
 'atividade_principal': [{'code': '49.30-2-04',
                          'text': 'Transporte rodoviário de mudanças'}],
 'atividades_secundarias': [{'code': '49.30-2-01',
                             'text': 'Transporte rodoviário de carga, exceto '
                                     'produtos perigosos e mudanças, '
                                     'municipal'},
                            {'code': '49.30-2-02',
                             'text': 'Transporte rodoviário de carga, exceto '
                                     'produtos perigosos e mudanças, '
                                     'intermunicipal, interestadual e '
                                     'internacional'},
                            {'code': '52.11-7-02', 'text': 'Guarda-móveis'}],
 'bairro': 'ZONA INDUSTRIAL (GUARA)',
 'billing': {'database': True, 'free': True},
 'capital_social': '93700.00',
 'cep': '71.250-710',
 'cnpj': '03.126.110/0001-92',
 'complemento': 'LOTE;',
 

In [55]:
def consulta_cnpj(numero_cnpj):
    url = f"https://receitaws.com.br/v1/cnpj/{numero_cnpj}"

    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers)
    
    dic_fornecedor = response.json()
    
    return dic_fornecedor

# Criar um dicionário vazio fora da função
dicionario = {}

# Chamar a função e atribuir o resultado ao dicionário
dicionario = consulta_cnpj('81243735001977')

# Imprimir o dicionário
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [56]:
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [57]:
dicionario['nome']

'POSITIVO TECNOLOGIA S.A.'

In [58]:
dicionario['telefone']

'(41) 3316-7700'

In [59]:
dicionario['email']

'fiscal@positivo.com.br'

In [60]:
dicionario['logradouro'] + ', ' + dicionario['numero'] + ' - ' + dicionario['bairro'] + ' - ' + dicionario['municipio'] + ' / ' + dicionario['uf']

'R JAVARI, 1255 - DISTRITO INDUSTRIAL I - MANAUS / AM'

In [61]:
# filtrando dos dados da coluna FAV
df_cnpj = df[df['NOME_PI'] != 'PROFISSIONAL DE SAUDE AUTONOMO']
df_cnpj = df[df['NOME_PI'] != 'ATENDIMENTO MEDICO-HOPITALR/FC']
df_cnpj = df[df['NOME_PI'] != 'IND-INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'IND - INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'INDENIZACAO DESPESAS.']
df_cnpj = df[df['NOME_PI'] != 'AUXILIO FINANCEIRO NÃO INDENIZÁVEL']
df_cnpj = df[df['NOME_PI'] != 'INRE - RESTITUICAO E RESSARCIMENTO']
df_cnpj = df[df['NOME_NDSI'] != 'OUTROS SERV.DE TERCEIROS PJ- PAGTO ANTECIPADO']
df_cnpj = df[df['CREDOR'] != '160129']

df_cnpj['CREDOR'].unique()

array(['01590728000264', '05462543000144', '08792763000124', ...,
       '13754000000129', '42923049000166', '98488040644'], dtype=object)

In [62]:
df_cnpj[df_cnpj['CREDOR'] == '160129']

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO


In [63]:
ano_atual = str(datetime.now().year)


df_corrente = df[df['ANO'] == ano_atual]
df_corrente.head(2)

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,01590728000264,MICROTECNICA INFORMATICA LTDA,2023-11-01,58,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905230,MAQUINAS E EQUIPAMENTOS ENERGETICOS,2023NE001319,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,DF,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,0.00,0.0,0.00,3720.6
1,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,05462543000144,EASYTECH INFORMATICA E SERVICOS LTDA,2023-10-31,59,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905237,EQUIPAMENTOS DE TIC - ATIVOS DE REDE,2023NE001318,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,DF,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,8070.92,0.0,8070.92,0.0


In [64]:
df['ANO'].unique()

array(['2023', '2022', '2021'], dtype=object)

# Pesquisas no DataFrame

In [65]:
# Criando um DataFrame para testes 
df_teste = df.copy()

## Empenhos com mais de 100 dias

### Empenhos da ND 30

In [66]:
# Empenhos da ND 30 com prazo de entrega superior a 100 dias

df_nd30_100dias = df_teste[(df_teste['A_LIQUIDAR'] != 0) & (df_teste['ND'] == '339030') & (df_teste['DIAS'] > 100) & (df_teste['TIPO_NE'] == 'ORDINARIO') ].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_nd30_100dias.sort_values(by='A_LIQUIDAR', ascending=False).reset_index(drop=True)

#df_nd30_100dias.to_excel('nd30_PA.xlsx')

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,30189974000126,WCS MATERIAIS E SERVICOS DE ENGENHARIA LTDA,2023-09-13,107,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903024,MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES,2023NE001164,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,2084.6,0.0,2084.6,0.0
1,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,65353401000170,COMERCIAL VENER LTDA,2023-09-13,107,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE001151,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,885.0,0.0,885.0,0.0
2,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2023,39242061000109,CENTRAL TINTAS CURVELO LTDA,2023-07-25,157,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903024,MATERIAL P/ MANUT.DE BENS IMOVEIS/INSTALACOES,2023NE000891,C4ENMILCAPE,CAPACITACAO DE PESSOAL,6012,171423,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160503,DECEX/GESTOR,C4EN,MIL,CAPE,552.9,0.0,552.9,0.0
3,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,49720354000108,MAXIMA LIMPEZA LTDA,2023-09-13,107,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE001149,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,495.0,0.0,495.0,0.0


### Empenhos ND 52

In [67]:
df_nd52_100dias = df_teste[(df_teste['A_LIQUIDAR'] != 0) & (df_teste['ND'] == '449052') & (df_teste['DIAS'] > 50) ].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_nd52_100dias.reset_index(drop=True)

#df_nd52_100dias.to_excel('nd52_PA.xlsx')

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,45169346000156,FRANCILANDIO THIERES DE CARVALHO SILVA,2023-08-10,141,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905212,APARELHOS E UTENSILIOS DOMESTICOS,2023NE001405,D5SAFUSASOC,ASSISTENCIA SOCIAL,0032,215846,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,PI,MG,167505,DGP,D5SA,FUS,ASOC,3964.69,0.0,3964.69,0.0
1,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,06351401000172,DIMAS INDUSTRIA DE MOVEIS LTDA,2023-09-29,91,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905242,MOBILIARIO EM GERAL,2023NE001612,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,167503,DECEX - GESTOR,C1EN,CON,DETM,6496.00,0.0,6496.00,0.0
2,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,40426345000126,CITE ELETRODOMESTICOS LTDA,2023-09-29,91,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905212,APARELHOS E UTENSILIOS DOMESTICOS,2023NE001611,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,GO,MG,167503,DECEX - GESTOR,C1EN,CON,DETM,14016.00,0.0,14016.00,0.0
3,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,47531706000143,ASTA MOBILI MOVEIS LTDA,2023-10-06,84,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905242,MOBILIARIO EM GERAL,2023NE001643,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,RS,MG,167503,DECEX - GESTOR,C1EN,CON,DETM,26372.50,0.0,26372.50,0.0
4,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,33618396000194,FABRICIO RACHADEL COSTA,2023-10-17,73,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905248,VEICULOS DIVERSOS,2023NE001662,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SC,MG,167503,DECEX - GESTOR,C1EN,CON,DETM,1084.90,0.0,1084.90,0.0
5,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,45769285000168,REDNOV FERRAMENTAS LTDA.,2023-10-17,73,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905248,VEICULOS DIVERSOS,2023NE001663,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SC,MG,167503,DECEX - GESTOR,C1EN,CON,DETM,890.68,0.0,890.68,0.0
6,167129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,81243735000903,POSITIVO TECNOLOGIA S.A.,2023-10-20,70,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE001682,C1ENCONDETM,CONCURSOS AS ESCOLAS MILITARES,0032,171502,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,BA,MG,167503,DECEX - GESTOR,C1EN,CON,DETM,7114.00,0.0,7114.00,0.0
7,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,05462543000144,EASYTECH INFORMATICA E SERVICOS LTDA,2023-10-31,59,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905237,EQUIPAMENTOS DE TIC - ATIVOS DE REDE,2023NE001318,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,DF,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,8070.92,0.0,8070.92,0.0
8,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,08792763000124,MEGA BYTE MAGAZINE LTDA,2023-10-31,59,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE001317,F8MODCXDEGE,ATENDER NEC DECEX MODERNIZACAO OPERACIONAL,6012,171418,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,SP,MG,160503,DECEX/GESTOR,F8MO,DCX,DEGE,26270.00,0.0,26270.00,0.0
9,160129,ESA,156M,MODERNIZACAO E TRANSFORMACAO ESTRATEG,2023,28811718000187,BRUNO DIGITAL COMERCIO DE MERCADORIA EM GERAL ..

### Consolidação ND 30 e ND 52

In [68]:
nd = ['449052', '339030' ]

cols = ['UG', 'ANO', 'CREDOR', 'NOME_CREDOR',
       'DATA', 'DIAS', 'ND', 'NDSI', 'NOME_NDSI', 
       'NE', 'PI', 'NOME_PI', 'UGR', 'NOME_UGR', 
       'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR','PAGO']

df_foco = df_teste.loc[df_teste['ND'].isin(nd) & (df_teste['DIAS'] > 60) & (df_teste['A_LIQUIDAR'] != 0), :].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_foco.reset_index(drop=True)

df_foco = df_foco.loc[:, cols]

# Baixando foco para Almox
df_foco.to_excel('Retorno_Almox.xlsx')

### Recursos Liquidados a pagar

In [69]:
df_restos = df_teste[(df_teste['ANO'] == '2022') & (df_teste['A_LIQUIDAR'] !=  0)].sort_values(by='A_LIQUIDAR', ascending=False)

df_restos.to_excel('Restos_a_pagar.xlsx')

In [75]:
cols = ['UG', 'SIGLA UG', 'ACAO', 'NOME_ACAO', 'ANO', 'CREDOR', 'NOME_CREDOR',
'DATA', 'DIAS', 'ND', 'NOME_ND', 'NDSI', 'NOME_NDSI', 'NE', 'PI', 
'NOME_PI', 'TIPO', 'TIPO_NE', 'NOME_UGR', 'A_LIQUIDAR', 
'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']

df_filtro = df_teste.loc[:,cols]
df_filtro = df_filtro[df_filtro['A_LIQUIDAR'] != 0]
df_filtro.sort_values(by='DIAS', ascending=False).head(2)

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,TIPO,TIPO_NE,NOME_UGR,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
2995,160129,ESA,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,2022,05691235000190,COPYUAI LOCACAO DE EQUIPAMENTOS LTDA,2022-10-06,449,339040,SERVIÇOS DE TECNOLOGIA DA INFORMAÇÃO E COMUNIC...,33904012,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,2022NE001087,C4ENMILCAPE,CAPACITACAO DE PESSOAL,RPNP,GLOBAL,DECEX/GESTOR,1598.00,0.0,1598.00,7990.00
2795,160129,ESA,219D,ADEQUACAO DE ATIVOS DE INFRAESTRUTURA,2022,17289559000186,FENARA & FENARA - ENGENHARIA E ADMINISTRACAO LTDA,2022-12-14,380,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903916,MANUTENCAO E CONSERV. DE BENS IMOVEIS,2022NE002482,B4OMOBMAQUA,AQUARTELAMENTO,RPNP,GLOBAL,DEC-GESTOR,155796.97,0.0,155796.97,204469.27


In [73]:
df_filtro.to_excel('empenhos_a_liquidar.xlsx')